In [6]:
import tensorflow as tf 
import matplotlib.pyplot as plt 
import numpy as np
import os 
from tqdm import tqdm
import cv2 as cv

In [5]:
data = []

In [4]:
for i in os.listdir(os.path.join("Data")):
    print(i)
    for j in tqdm(os.listdir(os.path.join("Data", i))):
        try: 
            img = 

Happy


100%|████████████████████████████████████████████████████████████████████████████| 903/903 [00:00<00:00, 179534.34it/s]


Neutral


100%|████████████████████████████████████████████████████████████████████████████████████████| 736/736 [00:00<?, ?it/s]


Sad


100%|██████████████████████████████████████████████████████████████████████████| 1042/1042 [00:00<00:00, 344837.05it/s]
